### 학습 목표
* 데이터 전처리를 배운다.
* 모델을 개선시킨다.
* 오류 처리된(2,3번 해결) 데이터를 활용한다.

대회 URL : https://www.dacon.io/competitions/official/235745/overview/description

### 데이터 살펴 보기

In [313]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [314]:
import pandas as pd

train = pd.read_csv('train_df_errno.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
age = pd.read_csv('age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1022, 14), (150, 2), (16, 23))

In [315]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   object 
 3   지역          2896 non-null   object 
 4   공급유형        2896 non-null   object 
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   object 
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2685 non-null   float64
 12  10분내버스정류장수  2892 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 339.5+ KB


In [316]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

In [317]:
# 컬럼명 변경
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수']

In [318]:
test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수']

In [319]:
train.describe()

,총세대수,전용면적,전용면적별세대수,공가수,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
count,2896.000000,2896.000000,2896.000000,2896.000000,2685.000000,2892.000000,2896.000000,2896.000000
mean,891.025207,44.559033,102.822514,12.877417,0.180261,3.703320,598.766575,558.315953
std,516.665211,32.000640,132.988590,10.653686,0.431074,2.666876,396.677644,437.015694
min,26.000000,12.620000,1.000000,0.000000,0.000000,0.000000,13.000000,13.000000
25%,514.000000,32.100000,14.750000,4.000000,0.000000,2.000000,277.000000,209.000000
50%,780.000000,39.905000,60.000000,11.000000,0.000000,3.000000,507.000000,481.000000
75%,1106.000000,51.292500,143.250000,20.000000,0.000000,4.000000,823.000000,763.000000
max,2568.000000,583.400000,1865.000000,55.000000,3.000000,20.000000,1798.000000,2550.000000


### 오류 데이터 확인

In [320]:
train.loc[((train['단지코드']=='C2675') | (train['단지코드']=='C2335') | (train['단지코드']=='C1327')), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


In [321]:
test.loc[ ((test['단지코드']=='C2675') | (test['단지코드']=='C2335') | (test['단지코드']=='C1327') ) , :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
579,C2675,512,아파트,경기도,국민임대,36.65,130,9.0,A,18476000,154790,0.0,3.0,1016.0
580,C2675,512,아파트,경기도,국민임대,46.90,44,9.0,A,34082000,232200,0.0,3.0,1016.0
581,C2675,512,아파트,경기도,국민임대,46.90,80,9.0,A,34082000,232200,0.0,3.0,1016.0
582,C2675,512,아파트,경기도,국민임대,46.90,50,9.0,A,34082000,232200,0.0,3.0,1016.0
583,C2675,512,아파트,경기도,국민임대,46.90,88,9.0,A,34082000,232200,0.0,3.0,1016.0
584,C2675,512,아파트,경기도,국민임대,47.40,30,9.0,A,34082000,232200,0.0,3.0,1016.0
585,C2675,512,아파트,경기도,국민임대,51.93,75,9.0,A,43819000,275900,0.0,3.0,1016.0
586,C2675,512,아파트,경기도,국민임대,51.93,15,9.0,A,43819000,275900,0.0,3.0,1016.0
891,C1327,98,아파트,대전광역시,행복주택,16.83,30,13.0,J,18800000,94000,NaN,17.0,98.0
892,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000,142000,NaN,17.0,98.0


### 오류 데이터 처리 확인
* 동일한 단지에 코드가 2개로 부여된 단지 코드 (3쌍) : ['C2085', 'C1397'], ['C2431', 'C1649'], ['C1036', 'C2675']
* (참고 사항) 주차면수는 하나의 단지임을 전제로 산정된 것이고 총세대수는 두 개 단지의 합계입니다. 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치

In [322]:
train.loc[train['단지코드']=='C2085',  "단지코드"] = 'N2085'
train.loc[train['단지코드']=='C1397',  "단지코드"] = 'N2085'

In [323]:
train.loc[train['단지코드']=='N2085', : ].shape

(14, 15)

In [324]:
train.loc[train['단지코드']=='N2085', : ]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2029,N2085,1339,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2030,N2085,1339,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2031,N2085,1339,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2032,N2085,1339,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2033,N2085,1339,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2034,N2085,1339,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2172,N2085,1339,아파트,서울특별시,공공임대(10년),51.96,63,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2173,N2085,1339,아파트,서울특별시,공공임대(10년),51.99,41,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2174,N2085,1339,아파트,서울특별시,공공임대(10년),59.93,315,0.0,A,61210000,695170,0.0,3.0,1590.0,1595.0
2175,N2085,1339,아파트,서울특별시,공공임대(분납),74.80,208,0.0,A,120611000,293490,0.0,3.0,1590.0,1595.0


### 오류 코드 변경
* C2431, C1649의 총세대수를 1047로 변경
* C2431, C1649의 등록차량대수를 1214로 변경
* 
C2431, C1649의 단지코드를 N2431로 변경

In [325]:
a = train.loc[train['단지코드']=='C2431', : ]
b = train.loc[train['단지코드']=='C1649', : ]

print(a.shape, b.shape)
print(a['총세대수'], b['총세대수'])
print(a['등록차량수'], b['등록차량수'])

(0, 15) (0, 15)
Series([], Name: 총세대수, dtype: int64) Series([], Name: 총세대수, dtype: int64)
Series([], Name: 등록차량수, dtype: float64) Series([], Name: 등록차량수, dtype: float64)


In [326]:
train.loc[ train['단지코드']=='C2431',  "총세대수" ] = 1047
train.loc[ train['단지코드']=='C1649',  "총세대수" ] = 1047

train.loc[ train['단지코드']=='C2431',  "등록차량수" ] = 1214
train.loc[ train['단지코드']=='C1649',  "등록차량수" ] = 1214

train.loc[ train['단지코드']=='C2431',  "단지코드" ] = 'N2431'
train.loc[ train['단지코드']=='C1649',  "단지코드" ] = 'N2431'

In [327]:
train.loc[ train['단지코드']=='N2431', : ].shape

(6, 15)

### 오류 코드 변경
* C1036의 총세대수를 1243로 변경
* C1036의 단지코드를 N1036로 변경

In [328]:
a = train.loc[train['단지코드']=='C2675', : ]
b = train.loc[train['단지코드']=='C1036', : ]
a.shape, b.shape

((0, 15), (0, 15))

In [329]:
train.loc[train['단지코드']=='C1036',  "총세대수"] = 1243
train.loc[train['단지코드']=='C1036',  "단지코드"] = 'N1036'

In [330]:
train.loc[ train['단지코드']=='N1036', : ].shape

(7, 15)

### 오류 3
3. 단지코드 등 기입 실수로 데이터 정제 과정에서 매칭 오류 발생  
 - (오류 내용) 단지코드 등 기입 실수로 총세대수가 주차면수에 비해 과하게 많거나 적은 경우가 발생하였고, 점검 결과 일부 데이터의 단지코드, 총세대수, 주차면수 등에서 오류가 검출되었습니다.
 - (발생 원인) 원천데이터 수집 과정에서 단지 코드 등이 잘못 기입되었고 이를 인지하지 못한 채 데이터 정제를 하여 오류가 발생하였습니다.
 - (관련 데이터) 아래와 같이 총 9개 단지에서 같은 문제가 확인되었습니다. 
※ 실수가 발생한 단지 코드 (9개 단지) : ['C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
 - C2335, C1327 단지는 테스트셋, 나머지는 트레인셋 입니다.

#### 오류 처리
* train 데이터 셋에 오류 발생 코드를 ERR04로 변경 후, 데이터 셋을 두개로 분리

In [331]:
train.loc[ train['단지코드']=='C1095',  "단지코드" ] = 'ERR04_1095'
train.loc[ train['단지코드']=='C2051',  "단지코드" ] = 'ERR04_2051'
train.loc[ train['단지코드']=='C1218',  "단지코드" ] = 'ERR04_1218'
train.loc[ train['단지코드']=='C1894',  "단지코드" ] = 'ERR04_1894'
train.loc[ train['단지코드']=='C2483',  "단지코드" ] = 'ERR04_2483'
train.loc[ train['단지코드']=='C1502',  "단지코드" ] = 'ERR04_1502'
train.loc[ train['단지코드']=='C1988',  "단지코드" ] = 'ERR04_1988'

In [332]:
train.loc[ train['단지코드'].str.contains('ERR'), :].shape

(0, 15)

In [333]:
train.loc[train['단지코드'].str.contains('ERR'), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


### 데이터 오류 처리 후, csv파일 만들기

In [334]:
train_df = train.copy()
train_df_errno = train.loc[ ~train['단지코드'].str.contains('ERR'), :]

In [335]:
train_df.to_csv('train')

### 결측치 처리

In [336]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [337]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       42
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [338]:
train['지역'].unique()

array(['경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도', '부산광역시',
       '제주특별자치도', '울산광역시', '충청북도', '전라남도', '경상북도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [339]:
test['지역'].unique()

array(['경기도', '부산광역시', '전라북도', '경상남도', '충청남도', '대전광역시', '제주특별자치도', '강원도',
       '울산광역시', '경상북도', '충청북도', '광주광역시', '전라남도', '대구광역시', '세종특별자치시'],
      dtype=object)

In [340]:
train['지역'].value_counts()

경기도        602
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       147
광주광역시      141
경상북도       111
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
울산광역시       34
세종특별자치시     24
Name: 지역, dtype: int64

In [341]:
test['지역'].value_counts()

경기도        259
강원도        106
부산광역시       88
경상남도        85
울산광역시       73
경상북도        58
대전광역시       56
충청북도        51
광주광역시       46
충청남도        41
전라북도        41
전라남도        37
대구광역시       36
제주특별자치도     26
세종특별자치시     19
Name: 지역, dtype: int64

In [342]:
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [343]:
test.loc[196,:]

단지코드             C2411
총세대수               962
임대건물구분             아파트
지역                경상남도
공급유형              국민임대
전용면적              46.9
전용면적별세대수           240
공가수                 25
자격유형               NaN
임대보증금         71950000
임대료              37470
10분내지하철수             0
10분내버스정류장수           2
단지내주차면수            840
Name: 196, dtype: object

In [344]:
grouped = test.groupby(['단지코드', '임대건물구분', '지역','공급유형'])
group1 = grouped.get_group( ('C2411', '아파트', '경상남도', '국민임대')  )
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0


In [345]:
group2 = grouped.get_group(('C2253', '아파트', '강원도', '영구임대'))
group2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0


In [346]:
test.loc[196, '자격유형'] = 'A'
test.loc[258, '자격유형'] = 'C'

In [347]:
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


In [348]:
print(train.자격유형.unique())
print(test.자격유형.unique())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['H' 'A' 'E' 'C' 'D' 'G' 'I' 'J' 'K' 'L' 'M' 'N']


In [349]:
mapping = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
           'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
           'K':11, 'L':12, 'M':13, 'N':14, 'O':15}
train['자격유형'] = train['자격유형'].map(mapping).astype(int)
test['자격유형'] =test['자격유형'].map(mapping).astype(int)

In [350]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [351]:
print(train.임대건물구분.unique())
print(test.임대건물구분.unique())

['아파트' '상가']
['아파트' '상가']


In [352]:
print(train.공급유형.unique())
print(test.공급유형.unique())

['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']
['국민임대' '영구임대' '임대상가' '공공임대(50년)' '공공임대(10년)' '공공임대(분납)' '행복주택']


In [353]:
mapping = {'국민임대':1, '영구임대':2, '임대상가':3, '공공임대(50년)':4, '공공임대(10년)':5, 
           '공공임대(분납)':6, '행복주택':7, '장기전세':8, '공공분양':9, '공공임대(5년)':10,
           'K':11, 'L':12, 'M':13, 'N':14, 'O':15}
train['공급유형'] = train['공급유형'].map(mapping).astype(int)
test['공급유형'] =test['공급유형'].map(mapping).astype(int)

In [354]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,1,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,1,46.81,30,14.0,1,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,1,46.90,112,14.0,8,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,1,46.90,120,14.0,8,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,1,51.46,60,14.0,8,43497000,296780,0.0,2.0,683.0


In [355]:
mapping_region = {'경기도':1, '부산광역시':2, '전라북도':3, '경상남도':4, '충청남도':5, '대전광역시':6, '제주특별자치도':7, 
                  '강원도':8, '울산광역시':9, '경상북도':10, '충청북도':11, '광주광역시':12, '전라남도':13, 
                  '대구광역시':14, '세종특별자치시':15, '서울특별시':16}
train['지역'] = train['지역'].map(mapping_region).astype(int)
test['지역'] = test['지역'].map(mapping_region).astype(int)

In [356]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,1,1,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,1,1,46.81,30,14.0,1,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,1,1,46.90,112,14.0,8,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,1,1,46.90,120,14.0,8,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,1,1,51.46,60,14.0,8,43497000,296780,0.0,2.0,683.0


In [357]:
import numpy as np

In [358]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [359]:
import mglearn
import sklearn
print(mglearn.__version__)
print(sklearn.__version__)

0.1.9
0.23.2


In [360]:
sel = ['총세대수', '전용면적', '전용면적별세대수', '공가수',  '단지내주차면수']
# sel = ['총세대수']
X = train[sel]
y = train['등록차량수']

In [361]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = LinearRegression().fit(X_train, y_train)  # 결측치가 있는 경우 넣으면 안 된다.
pred = model.predict(X_test)
pred

array([1716.26351813,  348.61048216,  783.88142282,  538.87547451,
        107.12306256,  717.12036055,  432.67696112,  276.12112804,
        644.72978349,  439.68033293,  143.51540176,  494.95910271,
        133.53925635,  399.75694049,  145.68262375,  364.96082204,
        -68.76232343,  355.56285592,   93.55394032,  686.73665198,
        174.40284374,  602.37853238, 1074.43450639,  659.61381571,
        270.0558529 ,  170.86078706,  758.50915285,  738.02907885,
        121.57417248,  319.38471893,  414.13886676,  206.39907595,
        712.75428592,  121.57417248,  310.86169569, 1295.92061276,
        600.57369125,  553.74222917,   85.63062696,  595.35919502,
        196.60418203,  705.1233294 ,  379.66124081,  703.09496894,
        436.57365574,  272.93553796,  123.76327336,  560.21270074,
        178.27701093,  554.51579091,   85.63062696,  581.69831777,
        119.71647457,  730.42571704,  214.45463952,   33.5383559 ,
        260.0721344 , 1525.31829281,  309.87672253,  656.68025

In [362]:
model.coef_, model.intercept_

(array([-0.15912372,  0.48962892,  0.0719852 , -6.98404745,  1.09596934]),
 100.64601235358606)

In [363]:
import pandas as pd

In [364]:
dict_dat = {'실제값': y_test, '예측값': pred, 
            '오차': y_test - pred, '오차절댓값': abs(y_test - pred), '오차제곱': (y_test - pred)**2, 
           '로그오차제곱': (np.log(y_test+1)-np.log(pred+1))**2}
dat = pd.DataFrame(dict_dat)
dat

<ipython-input-364-95c87d46e687>:3: RuntimeWarning: invalid value encountered in log
  '로그오차제곱': (np.log(y_test+1)-np.log(pred+1))**2}


,실제값,예측값,오차,오차절댓값,오차제곱,로그오차제곱
2309,2315.0,1716.263518,598.736482,598.736482,358485.374722,0.089467
1323,408.0,348.610482,59.389518,59.389518,3527.114829,0.024616
22,730.0,783.881423,-53.881423,53.881423,2903.207725,0.005058
1005,596.0,538.875475,57.124525,57.124525,3263.211413,0.010116
248,108.0,107.123063,0.876937,0.876937,0.769019,0.000065
...,...,...,...,...,...,...
2870,1204.0,1268.351289,-64.351289,64.351289,4141.088387,0.002707
232,316.0,1041.532496,-725.532496,725.532496,526397.403146,1.417305
610,498.0,627.217042,-129.217042,129.217042,16697.043916,0.053029
1355,480.0,392.013879,87.986121,87.986121,7741.557542,0.040813


In [365]:
mse = ((y_test - pred)**2).sum()/len(y_test - pred)
mse

42701.70133960169

In [366]:
mae = abs(y_test-pred).sum()/len(pred)
mae

149.1293341160225

In [367]:
rmse = mse**0.5
rmse
rmse = np.sqrt(mse)
rmse

206.64389983641348

### 7-3 주차 수요 예측 LASSO모델 적용해보기. MAE, MSE 구해보기

In [368]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

sel = ['총세대수', '전용면적', '전용면적별세대수', '공가수',  '단지내주차면수', '자격유형']
# sel = ['총세대수']
X = train[sel]
y = train['등록차량수']

normalize_X = MinMaxScaler().fit_transform(X)
ex_X = PolynomialFeatures(degree=2, include_bias=False).fit_transform(normalize_X)  # 데이터 feature 추가 생성

X_train, X_test, y_train, y_test = train_test_split(ex_X, y, random_state=0)

print('정규화, 추가 생성 : ', ex_X.shape, y.shape)
print(np.min(ex_X), np.max(ex_X))

정규화, 추가 생성 :  (2896, 27) (2896,)
0.0 1.0


In [369]:
X_train.shape

(2172, 27)

In [370]:
from sklearn.linear_model import Lasso # Lasso : L1, Ridge: L2
lasso = Lasso(alpha=0.1).fit(X_train, y_train)
pred = lasso.predict(X_test)
pred[0:10]

array([2019.65008666,  392.24733726,  800.88739133,  498.30046632,
        120.74690258,  718.94113017,  438.63449529,  317.25462096,
        658.14803242,  443.14276016])

In [371]:
lasso.coef_

array([-0.00000000e+00,  9.10177686e+02,  0.00000000e+00,  4.46380447e+02,
        1.64165505e+03, -1.30416775e+02, -4.55914610e+02, -4.74056617e+02,
        2.90154807e+01,  0.00000000e+00,  5.33340619e+02,  0.00000000e+00,
       -8.39513699e+02,  0.00000000e+00,  0.00000000e+00,  1.87726773e+03,
       -5.12612990e+02, -0.00000000e+00,  0.00000000e+00,  4.60423407e+02,
       -0.00000000e+00, -3.15036676e+02, -1.51132974e+03, -3.84044860e-01,
        1.09077078e+02,  4.13440767e+02, -0.00000000e+00])

In [372]:
print("학습용 데이터 세트 점수 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 데이터 세트 점수 : {:.2f}".format(lasso.score(X_test, y_test)))

학습용 데이터 세트 점수 : 0.83
테스트 데이터 세트 점수 : 0.82


In [373]:
mae = abs(y_test-pred).sum()/len(pred)
mae

131.9598976166298

In [374]:
mse = ((y_test - pred)**2).sum()/len(y_test - pred)
mse

35213.800851827385

In [375]:
rmse = mse**0.5
rmse

187.65340618232162

### 7-4 주차 수요 예측 Ridge 모델 적용해보기. MAE, MSE 구해보기

In [376]:
from sklearn.linear_model import Ridge # Lasso : L1, Ridge: L2
ridge = Ridge(alpha=0.1).fit(X_train, y_train)
pred = ridge.predict(X_test)
pred[0:10]

array([2029.11368446,  383.97130032,  808.49722234,  495.5838777 ,
        125.00886239,  725.17528719,  448.31966119,  310.8521135 ,
        656.60649668,  442.13558937])

In [377]:
ridge.coef_

array([   56.29867296,  1283.91055699,   -50.65904613,   464.99702448,
        1579.6380411 ,   -39.92848841,  -497.35236058,  -845.05433511,
         792.3598323 ,    24.79631946,   557.1279123 ,   -52.08829391,
        -755.80106246,   571.96177737,   457.0527057 ,  2061.10094671,
       -1789.36265055,  -706.30094106,   266.56530546,   284.44559235,
        -557.44649494,  -380.0955209 , -1576.58308144,    15.06000118,
         119.39185091,   547.27140981,   -60.3847873 ])

In [378]:
print("학습용 데이터 세트 점수 : {:.2f}".format(ridge.score(X_train, y_train)))
print("테스트 데이터 세트 점수 : {:.2f}".format(ridge.score(X_test, y_test)))

학습용 데이터 세트 점수 : 0.83
테스트 데이터 세트 점수 : 0.83


In [379]:
mae = abs(y_test-pred).sum()/len(pred)
mae

130.58525971696537

In [380]:
mse = ((y_test - pred)**2).sum()/len(y_test - pred)
mse

34519.63943419638

In [381]:
rmse = mse**0.5
rmse

185.79461626806193

In [382]:
test['등록차량수'] = pred
test['단지별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

ValueError: Length of values (724) does not match length of index (1022)

In [ ]:
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

In [ ]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

In [311]:
sub_df.to_csv('third_practice_regression_20210714.csv', index=False)
sub_df.head()

NameError: name 'sub_df' is not defined

In [31]:
import os
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 '01_데이터셋_살펴보기.ipynb',
 '02_second_model.ipynb',
 '03_third_model.ipynb',
 'age_gender_info.csv',
 'baseline_practice_0712.csv',
 'sample_submission.csv',
 'second_practice_randomforest_20210712.csv',
 'test.csv',
 'third_practice_randomforest_20210712.csv',
 'train.csv']